In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier

# 数据加载

In [2]:
train_data=pd.read_csv(r'../data/Titanic_Data/train.csv')
test_data=pd.read_csv(r'../data/Titanic_Data/test.csv')

# 数据预览

In [3]:
print(train_data.info())  #使用 info() 了解数据表的基本情况：行数、列数、每列的数据类型、数据完整度；
print('-'*30)
print(train_data.describe()) #使用 describe() 了解数据表的统计情况：总数、平均值、标准差、最小值、最大值等
print('-'*30)
# print(train_data.describe(include=['O'])) #使用 describe(include=[‘O’]) 查看字符串类型（非数字）的整体情况
# print('-'*30)
print(train_data.head())  #使用 head 查看前几行数据（默认是前 5 行）
# print('-'*30)
# print(train_data.tail())  #使用 tail 查看后几行数据（默认是最后 5 行）

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
------------------------------
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.00000

# 数据预处理
数据集中包括12列，代表12个特征；
所有的特征并不全是对分类有用的，同时有些特征存在nan缺失

In [4]:
# 使用平均年龄来填充年龄中的 nan 值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
# 使用票价的均值填充票价中的 nan 值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)
print(train_data['Embarked'].value_counts())

# 使用登录最多的港口来填充登录港口的 nan 值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)

S    644
C    168
Q     77
Name: Embarked, dtype: int64


# 特征选择
Passenger Id 为乘客编号，对分类没有作用，可以放弃；Name
为乘客姓名，对分类没有作用，可以放弃；Cabin 字段缺失值太多，可以放弃；Ticket 字
段为船票号码，杂乱无章且无规律，可以放弃。其余的字段包括：Pclass、Sex、Age、
SibSp、Parch 和 Fare，这些属性分别表示了乘客的船票等级、性别、年龄、亲戚数量以
及船票价格，可能会和乘客的生存预测分类有关系。

In [5]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
print(train_features)
train_labels = train_data['Survived']
test_features = test_data[features]

     Pclass     Sex        Age  SibSp  Parch      Fare Embarked
0         3    male  22.000000      1      0    7.2500        S
1         1  female  38.000000      1      0   71.2833        C
2         3  female  26.000000      0      0    7.9250        S
3         1  female  35.000000      1      0   53.1000        S
4         3    male  35.000000      0      0    8.0500        S
5         3    male  29.699118      0      0    8.4583        Q
6         1    male  54.000000      0      0   51.8625        S
7         3    male   2.000000      3      1   21.0750        S
8         3  female  27.000000      0      2   11.1333        S
9         2  female  14.000000      1      0   30.0708        C
10        3  female   4.000000      1      1   16.7000        S
11        1  female  58.000000      0      0   26.5500        S
12        3    male  20.000000      0      0    8.0500        S
13        3    male  39.000000      1      5   31.2750        S
14        3  female  14.000000      0   

特征值里有一些是字符串，这样不方便后续的运算，需要转成数值类型，比如 Sex 字段，
有 male 和 female 两种取值。我们可以把它变成 Sex=male 和 Sex=female 两个字段，
数值用 0 或 1 来表示。

In [6]:
dvec=DictVectorizer(sparse=False)
train_features=dvec.fit_transform(train_features.to_dict(orient='record'))
test_features=dvec.transform(test_features.to_dict(orient='record'))
print(train_features)

[[22.          0.          0.         ...  0.          1.
   1.        ]
 [38.          1.          0.         ...  1.          0.
   1.        ]
 [26.          0.          0.         ...  1.          0.
   0.        ]
 ...
 [29.69911765  0.          0.         ...  1.          0.
   1.        ]
 [26.          1.          0.         ...  0.          1.
   0.        ]
 [32.          0.          1.         ...  0.          1.
   0.        ]]


# 构造决策树

In [7]:
# clf = DecisionTreeClassifier(criterion='entropy')
clf = DecisionTreeClassifier(criterion='gini')  #分类标准对于不同的决策树算法
# 决策树训练
clf.fit(train_features, train_labels)

# 决策树预测
pred_labels = clf.predict(test_features)

# 得到决策树准确率
acc_decision_tree = round(clf.score(train_features, train_labels), 6)
print(u'score 准确率为 %.4lf' % acc_decision_tree)

score 准确率为 0.9820


# 交叉验证

In [8]:
from sklearn.model_selection import cross_val_score
# 使用 K 折交叉验证 统计决策树准确率
print(u'cross_val_score 准确率为 %.4lf' % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

cross_val_score 准确率为 0.7824
